In [11]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xml.etree.ElementTree as etree

# Load Data

In [12]:
# Field dimension on Instat 105 x 68 y
source_data = '/Users/matthiashugli/Dropbox/bucket/sample-data/23-10-2021--Switzerland--Credit-Suisse-Super-LeagueYoung-Boys-3-2-Lausanne-Sport20220220-76331-1ahfybv.xml'
large_data = '/Users/matthiashugli/Dropbox/bucket/sample-data/23-10-2021--Switzerland--Credit-Suisse-Super-LeagueYoung-Boys-3-2-Lausanne-Sport20220119-89784-gbgtk8.xml'
filtered_export = '/Users/matthiashugli/Dropbox/bucket/sample-data/13-02-2022--Switzerland--Credit-Suisse-Super-LeagueZurich-3-0-Lugano20220216-61331-brm4vi.xml'
#filtered_export = '/Users/matthiashugli/Dropbox/bucket/sample-data/13-02-2022--Switzerland--Credit-Suisse-Super-LeagueZurich-3-0-Lugano20220217-48763-1if7ja.xml'
test_source = '/Users/matthiashugli/Downloads/test.xml'

#data = pd.read_xml(test_source)

prstree = etree.parse(filtered_export)
root = prstree.getroot()

event_items = []
event_subitems = []
all_events = []
header = ['start', 'end', 'player', 'x_pos', 'y_pos', 'team', 'action', 'half']

for r in root.iter('ALL_INSTANCES'):
    for inst in r.iter('instance'):
    
        start = inst.find('start').text
        end = inst.find('end').text
        player = inst.find('code').text

        if inst.find('pos_x') != None:
            x_pos = inst.find('pos_x').text
            y_pos = inst.find('pos_y').text
        
        event_items = [start, end, player, x_pos, y_pos]
        
        for label in inst.iter('label'):
            labels = label.find('text').text
            event_items.append(labels)

        all_events.append(event_items)

df = pd.DataFrame(all_events, columns=header)

In [13]:
# Action variables
shot_outcome = ['Wide shot', 'Shot on target']
goal_situations = ['Assists', 'Goals', 'Free-kick attacks', 'penalty attack']
gk_actions = ['Wide shot (Goalkeepers)', 'Shot on target (saved)', 'Goals conceded']

In [14]:
shots = df[df['action'] == 'Shots'].reset_index()

passes_before_shots = df.reset_index()
passes_before_shots.rename(columns={'index': 'id'}, inplace=True)

for i in range(1, 5):
    passes_before_shots['action-' + str(i)] = passes_before_shots.id.shift(i)
    passes_before_shots['action+' + str(i)] = passes_before_shots.id.shift(-i)

melt_passes = pd.melt(passes_before_shots, id_vars=passes_before_shots.iloc[:,:8], value_vars=passes_before_shots.iloc[:,9:])
melt_passes = melt_passes[melt_passes['action'] == 'Shots']
melt_passes.dropna(subset=['value'], inplace=True)
melt_passes.merge(passes_before_shots, left_on='value', right_on='id')
melt_passes['value'] = melt_passes['value'].astype('int64')
merge_passes = melt_passes.merge(passes_before_shots, left_on='value', right_on='id').reset_index()

merge_passes['time_diff'] = (merge_passes['start_y'] - merge_passes['start_x'])
merge_passes['field_progression'] = merge_passes['x_pos_x'] - merge_passes['x_pos_y']

last_pass = merge_passes[(merge_passes['time_diff'] <= 0) & \
                         (~merge_passes['action_y'].isin(shot_outcome)) & \
                         (merge_passes['team_x'] == merge_passes['team_y'])] \
        .sort_values(by=['id_x', 'time_diff']).drop_duplicates(subset=['id_x'], keep='last')

shot_creation = shots.merge(last_pass[['id_x', 'x_pos_y', 'y_pos_y', 'action_y', 'time_diff']], how='left', left_on='index', right_on='id_x').drop(columns=['index', 'id_x', 'half'])

#print(shot_creation['time_diff'].mean() + (shot_creation['time_diff'].std() * - 1 ))
shot_creation

TypeError: 'DataFrame' objects are mutable, thus they cannot be hashed

In [24]:
goals = df[df['action'] == 'Goals'].reset_index()

passes_before_goals = df.reset_index()
passes_before_goals.rename(columns={'index': 'id'}, inplace=True)

for i in range(1, 5):
    passes_before_goals['action-' + str(i)] = passes_before_goals.id.shift(i)
    passes_before_goals['action+' + str(i)] = passes_before_goals.id.shift(-i)

melt_passes = pd.melt(passes_before_goals, id_vars=passes_before_goals.iloc[:,:8], value_vars=passes_before_goals.iloc[:,9:])
# melt_goals = melt_goals[melt_goals['action'] == 'Goals']
# melt_goals.dropna(subset=['value'], inplace=True)
# melt_goals.merge(passes_before_goals, left_on='value', right_on='id')
# melt_goals['value'] = melt_goals['value'].astype('int64')
# merge_goals = melt_goals.merge(passes_before_goals, left_on='value', right_on='id').reset_index()

# merge_goals['time_diff'] = np.where(merge_goals['start_y'] - merge_goals['start_x'] > 0, \
#                                     (merge_goals['start_y'] - merge_goals['start_x']) * -1, \
#                                     merge_goals['start_y'] - merge_goals['start_x'])
# merge_goals['field_progression'] = merge_goals['x_pos_x'] - merge_goals['x_pos_y']

# last_pass = merge_goals[(merge_goals['action_y'].isin(goal_situations)) & \
#                          (merge_goals['team_x'] == merge_goals['team_y'])] \
#         .sort_values(by=['id_x', 'time_diff'], ascending=True).drop_duplicates(subset=['id_x'], keep='last').reset_index()

# goal_creation = goals.merge(last_pass[['id_x', 'x_pos_y', 'y_pos_y', 'action_y', 'time_diff']], how='left', left_on='index', right_on='id_x').drop(columns=['index', 'id_x', 'half'])

# #print(shot_creation['time_diff'].mean() + (shot_creation['time_diff'].std() * - 1 ))
# goal_creation
passes_before_goals

TypeError: 'DataFrame' objects are mutable, thus they cannot be hashed